In [ ]:
# This Notebook is based on a clone of
# Joao Fernando Ferreira Goncalves
# original notebook that can be found here:
# https://github.com/Joaoffg/AISocIMP23/
#
# See original notebook here:
# https://colab.research.google.com/drive/1ya-NLUyfbs0lZJu1pQIkhoA4POtTsVkv?usp=sharing
#
# Changes to the original notebook was made, according to
# to talks with Joao, after Mads and I visited
# Rotterdam Univerity in January 2024.
#
# February 26, 2024
# sila

# Training a language model using Transformers

Training a language model is a more advanced form of machine learning,
therefore we need to install some libraries that are not default for Google Colab.
We also clone our GitHub repository as usual.

In [ ]:
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install accelerate
!git clone https://github.com/Joaoffg/AISocIMP23/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.8 MB/s eta 0:00:00
Cloning into 'AISocIMP23'...
remote: Enumerating objects: 284, done.
remote: Counting objects: 100% (190/190), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 284 (delta 57), reused 72 (delta 15), pack-reused 94
Receiving objects: 100% (284/284), 109.91 MiB | 22.27 MiB/s, done.
Resolving deltas: 100% (78/78), done.


After installing the libraries, we need to import them so we can use them in out code.

In [ ]:
from datasets import load_dataset
import re
import os
import torch
from transformers import LlamaTokenizer, LlamaConfig, LlamaModel, LlamaForCausalLM, Trainer, TrainingArguments

In [ ]:
%ls "/content/AISocIMP23/Week 4/Texts"

'ThesisDraft_MariaPalaciosBarea_622509 (1).txt'


This part loads all of the text data that you will use to train the language model.

In [ ]:
dataset = load_dataset("text",
                       data_dir="/content/AISocIMP23/Week 4/Texts")


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# What you must do:
# 1. Try to run the complete notebook based om this text.
# 2. Then re-run notebook, but now with more texts added, and see what
#     happens as information from these new texts books are queried.

This part tokenizes the dataset, which means that it converts all of the words into numbers that can be processed by the neural network.

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained('/content/AISocIMP23/Week 4/Token')

def chunk_examples(examples,chunk_lenght=128, min_chunk_lenght = 25):
    chunks = []
    for text in examples["text"]:
        tokenized = tokenizer(text,add_special_tokens=False)
        if len(tokenized.input_ids) > min_chunk_lenght:
            input_ids = [tokenizer.bos_token_id] + tokenized.input_ids + [tokenizer.eos_token_id]
            attention_mask = [1] + tokenized.attention_mask + [1]
            for i in range(0, len(tokenized.input_ids), chunk_lenght):
                cunk_input_ids = input_ids[i:i + chunk_lenght]
                cunk_att_mask = attention_mask[i:i + chunk_lenght]
                cur_chunk_len = len(cunk_input_ids)

                if  cur_chunk_len < chunk_lenght:
                    cunk_input_ids = cunk_input_ids + [tokenizer.eos_token_id]*(chunk_lenght - cur_chunk_len)
                    cunk_att_mask = cunk_att_mask + [0]*(chunk_lenght - cur_chunk_len)

                chunks += [{"input_ids":torch.tensor(cunk_input_ids),
                        "attention_mask": torch.tensor(cunk_att_mask),
                        "labels": torch.tensor(cunk_input_ids)}
                        #"raw":tokenizer.decode(cunk_input_ids)}

                        ]


    return {"chunks": chunks}


chunked_dataset = dataset.map(chunk_examples, batched=True, remove_columns=['text'])

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/1548 [00:00<?, ? examples/s]

Here we define our model architecture, we are using a LlaMa based model for this exercise. You can change the complexity factor below to make the model more simple or more complex.

In [ ]:
complexity_reduction=2

config = LlamaConfig(
    vocab_size = 32000,
    hidden_size= int(2048/complexity_reduction),
    intermediate_size = int(5120/complexity_reduction),
    num_hidden_layers = int(16/complexity_reduction),
    num_attention_heads = int(16/complexity_reduction),
    max_position_embeddings = 2048 ,
    rms_norm_eps = 1e-12
)

model = LlamaForCausalLM(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"LlaMa Model Size: {model_size/1000**2:.1f}M parameters")

tokenizer.pad_token = tokenizer.eos_token

LlaMa Model Size: 162.0M parameters


Here you define the training arguments. You can ignore most of them, they are default values, but you may want to tweak the batch_sizes and the learning rate.

In [ ]:
args = TrainingArguments(
    output_dir="erasmian-lm/medium",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    evaluation_strategy="no",
    eval_steps=5_000,
    logging_steps=5_000,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=0.0001,
    save_steps=5_000,
    fp16=True,
    save_strategy = "epoch", #save only latest model at end of epoch instead of 5k steps.
    save_total_limit = 1 # save only latest 3 epochs
)


In [ ]:
# Note: Try num_train_epochs=5, for improved performance.

This step trains the model, exciting!

In [ ]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    train_dataset=chunked_dataset['train']['chunks']

)

model = torch.compile(model)
trainer.train()

Step,Training Loss


TrainOutput(global_step=560, training_loss=5.395074898856027, metrics={'train_runtime': 127.3823, 'train_samples_per_second': 4.396, 'train_steps_per_second': 4.396, 'total_flos': 55589732352000.0, 'train_loss': 5.395074898856027, 'epoch': 1.0})

And here you can test how the model is actually performing by generating some text. You can write the start of the text between "" in the input_text field.

In [ ]:
from transformers import GenerationConfig

input_text= "Erasmus University is"

generation_config = GenerationConfig(
    temperature=1,
    top_p=0.95,
    top_k=50,
    repetition_penalty=1,
    do_sample=True,
    num_return_sequences=1
)

inputs = tokenizer(input_text, return_tensors="pt")
inputs = inputs.to("cuda:0")
model = model.to("cuda:0")
outputs = model.generate(**inputs, num_beams=1, do_sample=True, max_length=128)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['Erasmus University is and the of the of to colour, be the with the the prompt the “ resulting”.” as the a. The to the the its the are findings this, as a ( in the resulting this or was. The of the this additionally the “ findings biases which that on its from the additionally women biases a this the”, the which are on by the survey prompt a of the a this research to the the in the as the its andions, and the on resulting “’s this human that the they. This by the from biases women complet and “ to the the additionally complet. In in complet are']


In [ ]:
# Asking what Proshop is gives a nonsense answer
# 'Proshop is by, biases’s biases feel, and the women by a in the ( complet it from the the, as the social in theing that these of the other. For the to the are of the its in this and the in the which social” of the this ( additionally the the “ ( social its it is of the’s that GPT-3 ( these prompt that “ are the ( as the a ( was of the the (, theed the was “ of more this “ an to the socialed complet are a the social.']
